*Bearbeitet von Anh Thi Pham*

Der Python Code, sowie die Dokumentation wurde in *jupyter notebook* geschrieben.Die Ergebnisse zu dieser Übung sind unter [Ausgabe der Ergebnisse](#ausgabe-der-ergebnisse) zu finden.

# Theorie
## Abschätzung der Stoffmengenströme
Um die Stoffmengenströme innerhalb der Kolonne abzuschätzen wird ein äquimolarer Stofftransport angenommen. Aus der Gesamtbilanz lässt sich bei bekannten Feed- und Kopfstrom der Sumpfstrom berechnen.  

\begin{equation}
\dot{S}=\dot{F}-\dot{K} 
\end{equation}

Aus dem Feedstrom und dem Dampfanteil $f$ lässt sich der flüssige und dampfförmige Feedstrom berechnen

\begin{equation}
\dot{F}=(1-f) \dot{F}^L + f \dot{F}^G 
\end{equation}

Zur Bestimmung aller flüssigen und gasförmigen Gesamtstoffströme oberhalb der Feedstufe, also $j < n$ wird eine Gesamtbilanz um den Kopf der Kolonne gebildet.

\begin{equation}
\dot{L}_j = \nu \dot{K}
\end{equation}

\begin{equation}
\dot{G}_{j+1}= \dot{K} + \dot{L}_j 
\end{equation}

Darin ist $\nu$ das Rücklaufverhältnis.  
Für alle Gesamtstoffströme unterhalb der Feedstufe, also $j > n$, folgt aus einer Bilanz um die Feedstufe folgende Gleichungen:

\begin{equation}
\dot{L}_{j}= \dot{L}_{n-1} + \dot{F}^L 
\end{equation}

\begin{equation}
\dot{G}_{j+1}= \dot{G}_{n} - \dot{F}^G
\end{equation}

Die obigen Gleichungen sind in der Funktion *estimate_flux_cmo* implementiert, die die Arrays $G_j$ und $L_j$ als Spaltenvektoren zurückgibt.

| $G_j$   | $L_j$ |
|---------|------ |
| $K$     | $L_0$ |
| $G_1$   | $L_1$ |
| $G_2$   | $L_2$ |
| $...$   | $...$ |
|$G_{n-1}$| $S$   |

## Berechnung der Komponentenströme
Für die Berechnung der Komponentenströme wird angenommen, dass die austretenden Gas- und Flüssigströme einer Trennstufe im thermodynamischen Gleichgewicht zueinander stehen. Mit dieser Annahme können die flüssigen Komponentenströme aus den gasförmigen Komponentenströmen berechnt werden.

\begin{equation} \label{l_from_v}
l_{ji} = \dot{L}_i x_{ji}= \frac{\dot{L}_i}{\dot{G}_i K_{ji}} \dot{G}_j y_{ji} = A_{ji} v_{ji}
\end{equation}

Darin ist $A_{ji}$ der **Absorptionskoeffizient**. Für alle Trennstufen ist der Absorptionskoeffizient wie folgt definiert:

\begin{equation}
A_{ji} = \frac{\dot{L}_i}{\dot{G}_i K_{ji}}
\end{equation}

Wird ein Totalkondensator eingesetzt, entpsricht der Absorptionskoeffizient für $j=0$ dem Rücklaufverhältnis.

\begin{equation}
A_{0i} = \frac{\dot{L}_0}{\dot{K}}
\end{equation}

Auch bei einem Totalverdampfer liegt keine Trennstufe vor und der Aktivitätskoeffizient für $j=n+1$ wird wie folgt berechnet:

\begin{equation}
A_{n+1,i} = \frac{\dot{S}}{\dot{G}_{n+1}}
\end{equation}


$K_{ji}$ ist die **thermodynamische Gleichgewichtskonstante** der Komponente i auf der Stufe j. Diese lässt sich aus dem Rault-Dalton Gesetz ermitteln.

\begin{equation}
K_{ji}= \frac{y_{ji}}{x_{ji}} = \frac{p_{ji}^*}{p}
\end{equation}

Hier ist $p_{ji}^*$ Der Dampfdruck der Komponente i auf der Stufe j, welcher sich mit der Wagner-Gleichung in der 2,5-5-Form berechnen lässt. Mit der Funktion *calc_p_vap_trays* wird der Dampfdruck jeder Komponente auf jeder Kolonnenstufe berechnet. Das Ergebnis ist ein zweidimensionales $n \times m$ Array. 

Um nun alle **Komponentenströme** zu berechnen wird um jede Kolonnenstufe eine Bilanz erstellt und alle Flüssig-Komponentenströme mit Gleichung $(\ref{l_from_v})$ durch Gas-Komponentenströme ersetzt. Alle Bilanzen lassen sich für jede Komponente i in einer Matrixschreibweise schreiben.

\begin{equation} \label{matrix_short}
A_i \cdot v_i = L_i
\end{equation}

Ausgeschriebn ist Gleichung $(\ref{matrix_short})$ 

\begin{equation}
 \begin{pmatrix}
   -(A_{0i}+1) & 1           & 0           & 0           & \cdots & \cdots & 0 \\
   A_{0i}      & -(A_{1i}+1) & 1           & 0           & \cdots & \cdots & 0 \\
   0           & A_{1i}      & -(A_{2i}+1) & 1           & \cdots & \cdots & 0 \\
   0           & 0           & A_{2i}      & -(A_{3i}+1) & 1      & \cdots & 0 \\
   \vdots      & \vdots      & \vdots      & \vdots      & \ddots      & \vdots & 0 \\
   0           & 0           & 0           & 0           & \cdots      & A_{ni} & -(A_{n+1,i}+1)
 \end{pmatrix}
 \cdot
 \begin{pmatrix}
  d_i \\
  v_{1i}      \\
  v_{2i}      \\
  \vdots      \\
  \vdots      \\
  v_{n+1,i}           
 \end{pmatrix} =
 \begin{pmatrix}
  0 \\
  \vdots      \\
  v_{Fi}      \\
  l_{Fi}      \\
  \vdots      \\
  0
 \end{pmatrix}
\end{equation}

Die Gas-Komponentenströme $v_i$ können mit Hilfe der inversen Matrix von $A_i$ ermittelt werden.

\begin{equation}
v_i = A_i^{-1} \cdot L_i 
\end{equation}

$v_{ji}$ setzt sich als Matrix aus $v_i$ für alle Komponenten zusammen, wobei $v_i$ die einzelnen Spaltenvektoren sind. Mit Gleichung $(\ref{l_from_v})$ kann nun auch $l_{ji}$ aus $v_{ji}$ berechnet werden.   

Nun kann die **Zusammensetzung** der Flüssigphase berechnet werden; die Berechnung der Zusammensetzung der Gasphase erfolgt analog.

\begin{equation}
x_{ji} = \frac{l_{ji}}{\displaystyle\sum_{i=1}^{m} l_{ji}}
\end{equation}

Die Umsetzung der beschriebenen Berechnungen in Python sind im Abschnitt [Berechnungen](#berechnungen) zu finden.

Dieser Block wird genutz um Gleichungen automatisch zu nummerieren:

In [1]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});

<IPython.core.display.Javascript object>

In [2]:
%config Completer.use_jedi = False  # to make autocomplete work again

# Pyton Code

In [3]:
import numpy as np
import pandas as pd
from IPython.core.display import HTML, Markdown

import project_path
from tvt_lib.read_chem_properties import get_component_data
from tvt_lib.rectification import estimate_flux_cmo, calc_p_vap_trays, calc_L_i, calc_v_ji

In [4]:
# setting format options for pandas and numpy
pd.set_option('display.notebook_repr_html', True)

def _repr_latex_(self):
    return self.to_latex()

pd.DataFrame._repr_latex_ = _repr_latex_

pd.set_option("display.float_format", '{:.3f}'.format)
np.set_printoptions(precision=3, suppress=True)

## Angabe aller Parameter
- Naturkonstanten: universelle Gaskonstante
- Komponenten: Namen, Anzahl
- Kolonne: Kolonnenstufen, Totalkondensator (ja/nein), Totalverdampfer (ja/nein)
- Betriebsbedingungen: Temperatur, Druck
- Feed: Feedstrom, Feedzusammensetzung, Feedtempertur, Dampfanteil, Stufe der Feedzugabe
- Kopfprodukt: Kopfstrom, Rücklaufverhältnis

In [5]:
R = 8.314  # universelle Gaskonstante [J/(mol*K)]

comp_name = np.array(["Methanol", "Ethanol", "n-Propanol"])
m = comp_name.size  # Anzahl Komponenten

n = 5  # Anzahl Kolonnenstufen (Kondensator und Verdampfer werden hinzugezählt; Entspricht Anzahl Böden + 2)
total_condenser = True
total_reboiler = False

T_0 = 310.93  # Anfangstemperatur [K]
T = T_0 * np.ones(n)  # Temperaturarray für alle Kolonnenstufen
p_ges = 1.013  # Gesamtdruck [bar]

F = 100.0  # Feedstrom [mol/h]
z_F = np.array([1 / 3, 1 / 3, 1 / 3])  # Feedzusammensetzung
T_F = T_0  # Feedtemperatur
n_feed = 2  # Stufe der Feedzugabe
dampfanteil = 0  # Dampfanteil Feed [0, 1]

K = 50  # Kopfstrom [mol/h]
nue = 1  # Rücklaufverhältnis

## Stoffdaten einlesen
Gebraucht werden die allgemeinen Daten, sowie die Daten zur Berechnung des Dampfdrucks.

In [6]:
# allgemein
columns_general = ["Substance German", "Formula", "Tc", "Pc", "Dh0f"]
df_general = get_component_data(comp_name, 'allgemein', columns_general)
display(Markdown('**Allgemeine Stoffdaten**'))
display(df_general)
print()


# Dampfdruck
columns_p_vap = ["Substance German", "Formula", "A", "B", "C", "D"]
df_p_vap = get_component_data(comp_name, 'Dampfdruck', columns_p_vap)
display(Markdown('**Koeffizienten zur Dampfdruckberechnung (Wagner-Gleichung)**'))
display(df_p_vap)

**Allgemeine Stoffdaten**

,Formula,Tc,Pc,Dh0f
Substance German,,,,
Methanol,CH3OH,513.38,82.159,-201160
Ethanol,C2H5OH,513.90,61.480,-234800
n-Propanol,CH3-CH2-CH2-OH,536.75,51.750,-255200


**Koeffizienten zur Dampfdruckberechnung (Wagner-Gleichung)**

,Formula,A,B,C,D
Substance German,,,,,
Methanol,CH3OH,-8.727,1.450,-2.772,-0.724
Ethanol,C2H5OH,-8.338,0.087,-3.306,-0.260
n-Propanol,CH3-CH2-CH2-OH,-8.607,2.174,-8.047,3.692


## <a name="berechnungen"></a>Berechnungen

In [7]:
# Abschätzung der Molströme mit der Annahme eines äquimolaren Stofftransports
G_j, L_j = estimate_flux_cmo(n, F, K, nue, dampfanteil, n_feed)

# Berechnung der Dampfdrücke der für jeden Kolonnenboden    
df_properties = df_general.merge(df_p_vap, on=['Substance German', 'Formula'])
p_vap_trays = calc_p_vap_trays(T, comp_name, df_properties)

# Berechnung der Gleichgewichtskoeffizienten
K_ji = p_vap_trays / p_ges

# Berechnung der Absorptionskoeffizienten
A_ji = (L_j / G_j) / K_ji
if total_condenser:
    A_ji[0, :] = nue
if total_reboiler:
    A_ji[n - 1, :] = L_j[n - 1] / G_j[n - 1]

# Befüllen des Lösungsvektors L_i
L_i = calc_L_i(F, z_F, n_feed, dampfanteil, T_F, p_ges, comp_name, df_properties, n, m)

# Berechnung Komponentenströme
v_ji = calc_v_ji(A_ji, L_i, n, m)
V_j_calc = np.sum(v_ji, axis=1)[:, np.newaxis]
l_ji = v_ji * A_ji
L_j_calc = np.sum(l_ji, axis=1)[:, np.newaxis]

# print(l_ji[-1]+v_ji[0]) # Überprüft Komponentenbilanz; muss z_F*F_Feed entsprechen

# Berechnung der Zusammensetzung von Gas- und Flüssigphase
y_ji = v_ji / V_j_calc
x_ji = l_ji / L_j_calc


## <a name='zusammenfassung-der-ergebnisse'></a> Zusammenfassung der Ergebnisse

In [8]:
def add_prefix_list(prefix, l):
    return [prefix + item for item in l]


df_G_j = pd.DataFrame(data=G_j, columns=['G Flux estimate [mol/h]'])
df_L_j = pd.DataFrame(data=L_j, columns=['L Flux estimate[mol/h]'])
df_V_j_calc = pd.DataFrame(data=V_j_calc, columns=['G Flux calc[mol/h]'])
df_L_j_calc = pd.DataFrame(data=L_j_calc, columns=['L Flux calc[mol/h]'])
df_v_ji = pd.DataFrame(data=v_ji, columns=add_prefix_list('v_', comp_name))
df_l_ji = pd.DataFrame(data=l_ji, columns=add_prefix_list('l_', comp_name))
df_y_ji = pd.DataFrame(data=y_ji, columns=add_prefix_list('y_', comp_name))
df_x_ji = pd.DataFrame(data=x_ji, columns=add_prefix_list('x_', comp_name))


# Ausgabe der Anfangsparameter
display(Markdown('### <a name="ausgabe-der-ergebnisse"></a> Ausgabe der Ergebnisse'))
display(Markdown('**Parameter Rektifikation**'))
print(f"Namen der Komponenten:\t {comp_name}")
print("Anzahl Komponenten:\t", m)
print()
print("Anzahl Kolonnenstufen:\t", n)
print(f"Totalkondensator:\t {'Ja' if total_condenser else 'Nein'} ")
print(f"Totalverdampfer:\t {'Ja' if total_reboiler else 'Nein'} ")
print()
print(f"Kolonnentemperatur [K]:\t {T}")
print("Gesamtdruck Kolonne [bar]:\t", p_ges)
print()
print("Feedstrom [mol/h]:\t", F)
print(f"Feedzusammensetzung:\t", z_F)
print("Feedtemperatur [K]:\t", T_F)
print("Stufe der Feedzugabe:\t", n_feed)
print("Dampfanteil im Feed:\t", dampfanteil)
print()
print("Kopfstrom [mol/h]:\t", K)
print("Rücklaufverhältnis:\t", nue)
print()

display(Markdown('**Stoffströme - Erste Abschätzung**'))
display(df_G_j.join(df_L_j))
print()

display(Markdown('**Stoffströme - Berechnet**'))
display(df_V_j_calc.join(df_L_j_calc))
print()

display(Markdown('**Komponentenströme**'))
display(df_v_ji)
display(df_l_ji)
print()

display(Markdown('**Zusammensetzung**'))
display(df_y_ji)
display(df_x_ji)


### <a name="ausgabe-der-ergebnisse"></a> Ausgabe der Ergebnisse

**Parameter Rektifikation**

Namen der Komponenten:	 ['Methanol' 'Ethanol' 'n-Propanol']
Anzahl Komponenten:	 3

Anzahl Kolonnenstufen:	 5
Totalkondensator:	 Ja 
Totalverdampfer:	 Nein 

Kolonnentemperatur [K]:	 [310.93 310.93 310.93 310.93 310.93]
Gesamtdruck Kolonne [bar]:	 1.013

Feedstrom [mol/h]:	 100.0
Feedzusammensetzung:	 [0.333 0.333 0.333]
Feedtemperatur [K]:	 310.93
Stufe der Feedzugabe:	 2
Dampfanteil im Feed:	 0

Kopfstrom [mol/h]:	 50
Rücklaufverhältnis:	 1



**Stoffströme - Erste Abschätzung**

,G Flux estimate [mol/h],L Flux estimate[mol/h]
0,50.000,50.000
1,100.000,50.000
2,100.000,150.000
3,100.000,150.000
4,100.000,50.000


...

,v_Methanol,v_Ethanol,v_n-Propanol
0,2.122,0.541,0.080
1,4.244,1.082,0.160
2,8.837,3.951,1.403
3,10.732,4.568,1.505
4,19.728,10.404,4.027


,l_Methanol,l_Ethanol,l_n-Propanol
0,2.122,0.541,0.080
1,6.715,3.410,1.323
2,41.943,37.361,34.758
3,50.939,43.197,37.281
4,31.211,32.792,33.253


**Zusammensetzung**

,y_Methanol,y_Ethanol,y_n-Propanol
0,0.774,0.197,0.029
1,0.774,0.197,0.029
2,0.623,0.278,0.099
3,0.639,0.272,0.090
4,0.578,0.305,0.118


,x_Methanol,x_Ethanol,x_n-Propanol
0,0.774,0.197,0.029
1,0.587,0.298,0.116
2,0.368,0.328,0.305
3,0.388,0.329,0.284
4,0.321,0.337,0.342


Aus diesen Ergebnissen wird ersichtlich, dass mit den angegeben Parametern (vor allem Temperatur und Druck) kein Kopfstom von 50 mol/h erreicht werden kann.

## Ausgabe als Excel Datei

In [9]:
with pd.ExcelWriter('output.xlsx') as writer:
    df_G_j.join([df_L_j, df_V_j_calc, df_L_j_calc]).to_excel(writer, sheet_name='total_flux', index=False)
    df_v_ji.join(df_l_ji).to_excel(writer, sheet_name='comp_flux', index=False)
    df_y_ji.join(df_x_ji).to_excel(writer, sheet_name='composition', index=False)